In [1]:
import tensorflow as tf
import numpy as np
import h5py
import matplotlib.pyplot as plt

import models.graph_nn as grap

In [2]:
%load_ext autoreload
%autoreload 2

# data preprocessing

In [3]:
def normalized_adjacency(A):

    D = np.array(np.sum(A, axis=2), dtype=np.float32) # compute outdegree (= rowsum)
    D = np.nan_to_num(np.power(D,-0.5), posinf=0, neginf=0) # normalize (**-(1/2))
    D = np.asarray([np.diagflat(dd) for dd in D]) # and diagonalize
    
    return np.matmul(D, np.matmul(A, D))

In [4]:
def make_adjacencies(particles):
    real_p = particles[:,:,0] > 0
    adjacencies = (real_p[:,:,np.newaxis] * real_p[:,np.newaxis,:]).astype('int')
    return adjacencies

# load data

In [5]:
filename = '/home/kinga/dev/datasamples/L1_anomaly_challenge/background_training_1M.h5'
ff = h5py.File(filename, 'r')
particles = np.asarray(ff.get('Particles'))

In [6]:
particles.shape

(1000000, 19, 4)

In [7]:
nodes_n = particles.shape[1]
feat_sz = particles.shape[2]

In [8]:
# features: array([b'Pt', b'Eta', b'Phi', b'Class'], dtype='|S5')
particles_train = particles[:3000]

In [9]:
A = make_adjacencies(particles_train)
A_tilde = normalized_adjacency(A)

/home/kinga/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in power
  after removing the cwd from sys.path.


# build model: Autoencoder

In [10]:
gnn = grap.GraphAutoencoder(nodes_n=nodes_n, feat_sz=feat_sz, activation=tf.nn.tanh)
gnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), run_eagerly=True)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input_features (InputLa [(None, 19, 4)]      0                                            
__________________________________________________________________________________________________
encoder_input_adjacency (InputL [(None, 19, 19)]     0                                            
__________________________________________________________________________________________________
graph_convolution (GraphConvolu (None, 19, 3)        12          encoder_input_features[0][0]     
                                                                 encoder_input_adjacency[0][0]    
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 19, 2)        6           graph_convolution[0][0

# train model

In [11]:
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, verbose=1)]
gnn.fit((particles_train, A_tilde), A, epochs=100, batch_size=128, validation_split=0.25, callbacks=callbacks)

Epoch 1/100
12/12 [==============================] - 0s 21ms/step - val_loss: 0.6806
Epoch 2/100
12/12 [==============================] - 0s 18ms/step - val_loss: 0.6797
Epoch 3/100
12/12 [==============================] - 0s 18ms/step - val_loss: 0.6796
Epoch 4/100
12/12 [==============================] - 0s 17ms/step - val_loss: 0.6795
Epoch 5/100
12/12 [==============================] - 0s 18ms/step - val_loss: 0.6795
Epoch 6/100
12/12 [==============================] - 0s 17ms/step - val_loss: 0.6795
Epoch 7/100
12/12 [==============================] - 0s 16ms/step - val_loss: 0.6795
Epoch 8/100
 9/12 [=====================>........] - ETA: 0s
Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
12/12 [==============================] - 0s 17ms/step - val_loss: 0.6795
Epoch 9/100
12/12 [==============================] - 0s 16ms/step - val_loss: 0.6795
Epoch 10/100
12/12 [==============================] - 0s 17ms/step - val_loss: 0.6795
Epoch 11/100
12/12 [=

Epoch 77/100
12/12 [==============================] - 0s 17ms/step - val_loss: 0.6795
Epoch 78/100
11/12 [==========================>...] - ETA: 0s
Epoch 00078: ReduceLROnPlateau reducing learning rate to 6.103515488575795e-07.
12/12 [==============================] - 0s 18ms/step - val_loss: 0.6795
Epoch 79/100
12/12 [==============================] - 0s 16ms/step - val_loss: 0.6795
Epoch 80/100
12/12 [==============================] - 0s 16ms/step - val_loss: 0.6795
Epoch 81/100
12/12 [==============================] - 0s 17ms/step - val_loss: 0.6795
Epoch 82/100
12/12 [==============================] - 0s 17ms/step - val_loss: 0.6795
Epoch 83/100
12/12 [==============================] - ETA: 0s
Epoch 00083: ReduceLROnPlateau reducing learning rate to 3.0517577442878974e-07.
12/12 [==============================] - 0s 17ms/step - val_loss: 0.6795
Epoch 84/100
12/12 [==============================] - 0s 16ms/step - val_loss: 0.6795
Epoch 85/100
12/12 [==============================] -

# predict

In [12]:
particles_test = particles[2000:3000]
A_test = make_adjacencies(particles_test)
A_tilde_test = normalized_adjacency(A_test)

/home/kinga/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in power
  after removing the cwd from sys.path.


In [15]:
z, A_pred_probs = gnn((particles_test, A_tilde_test))

In [16]:
A_pred = (A_pred_probs > 0.5).numpy().astype('int') 